In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 수용가별로 파일 분리하는 코드

- 너무 짧은 수용가는 제거한다.

In [3]:
total_df = pd.read_csv("원주20200401_20200630.csv")
total_df = total_df.append(pd.read_csv("원주20200701_20200930.csv"))
total_df = total_df.append(pd.read_csv("원주20201001_20201231.csv"))
total_df = total_df.append(pd.read_csv("원주20210101_20210228.csv"))
total_df = total_df.append(pd.read_csv("원주20210301_20210326.csv"))

people = total_df['수용가번호'].drop_duplicates()

In [ ]:
#type(df_someone['측정시간'].iloc[0])
sz = []
for p in people:
  df_someone = total_df[total_df['수용가번호']==p]
  # 앞부분 NA 제거
  for drop_index in range(df_someone.shape[0]):
    if not pd.isna(df_someone.loc[:, '적산유량보정값'].iloc[drop_index]):
      break
  df_someone = df_someone.iloc[drop_index:]
  sz.append(df_someone.shape[0])

  # 데이터가 너무 짧으면 사용하지 않음
  if df_someone.shape[0] < 2000:
    continue
  
  # 정상이라면 0, 정상이 아니라면 (누수, 고장 전부) 1
  df_someone['상태코드'] = (df_someone['상태코드'] != '정상').astype(int)
  t = 0.0
  nxt = 1
  features = ['미터배터리', '적산유량보정값', '구간유량보정값', '지침값오프셋', 'snr']
  new_df = df_someone.iloc[:1] # 새로 만들어지는 사람별 데이터
  receive_check = [1]
  receive_cnt = 0
  # 수신되지 않은 시간에 na를 채우고 또 1시간 단위에서 어긋난 데이터를 보간하여 채운다.
  for i in range(1, df_someone.shape[0]):
    t += df_someone.iloc[i]['시간간격']
    receive_cnt += 1
    while nxt <= t:
      if abs(nxt - t) < 0.02:
        new_df = new_df.append(df_someone.iloc[i:i+1])
        receive_check.append(receive_cnt)
        receive_cnt = 0
      else:
        before_row = new_df.iloc[-1]
        one_row = df_someone.iloc[i]
        if (nxt + 1 < t): # 긴 시간동안 수신되지 않으면 보간하지 않고 na를 채운다
          for f in features:
            one_row[f] = np.nan
        else:             # 1시간 간격을 어긋난 수준에서 비어있다면 보간하여 데이터를 채운다
          m = t - nxt + 1
          for f in features:
            one_row[f] = 1/m * one_row[f] + (t - nxt)/m * before_row[f]
          # 보간 후 구간 유량보정값을 올바르게 한번 더 수정한다.
          one_row['구간유량보정값'] = one_row['적산유량보정값']-before_row['적산유량보정값']
        receive_check.append(receive_cnt)
        receive_cnt = 0
        new_df = new_df.append(one_row)
      nxt += 1
  new_df.loc[:, '순번'] = list(range(0, new_df.shape[0]))
  re_ch = pd.DataFrame({'순번': new_df.loc[:, '순번'], '수신횟수':receive_check},)

  new_df = new_df.join(re_ch.set_index('순번'), on='순번')
  new_df.to_csv("사람별\\"+str(p)+".csv", sep=',', na_rep='NaN')

In [ ]:
##
#누수만 비정상으로 하기
###
sz = []
for p in people:
  df_someone = total_df[total_df['수용가번호']==p]
  # 앞부분 NA 제거
  for drop_index in range(df_someone.shape[0]):
    if not pd.isna(df_someone.loc[:, '적산유량보정값'].iloc[drop_index]):
      break
  df_someone = df_someone.iloc[drop_index:]
  sz.append(df_someone.shape[0])

  # 데이터가 너무 짧으면 사용하지 않음
  if df_someone.shape[0] < 2000:
    continue
  
  # 누수라면 1, 그 외 0
  df_someone['상태코드'] = (df_someone['상태코드'] == '누수').astype(int)
  t = 0.0
  nxt = 1
  features = ['미터배터리', '적산유량보정값', '구간유량보정값', '지침값오프셋', 'snr']
  new_df = df_someone.iloc[:1] # 새로 만들어지는 사람별 데이터
  receive_check = [1]
  receive_cnt = 0
  # 수신되지 않은 시간에 na를 채우고 또 1시간 단위에서 어긋난 데이터를 보간하여 채운다.
  for i in range(1, df_someone.shape[0]):
    t += df_someone.iloc[i]['시간간격']
    receive_cnt += 1
    while nxt <= t:
      if abs(nxt - t) < 0.02:
        new_df = new_df.append(df_someone.iloc[i:i+1])
        receive_check.append(receive_cnt)
        receive_cnt = 0
      else:
        before_row = new_df.iloc[-1]
        one_row = df_someone.iloc[i]
        if (nxt + 1 < t): # 긴 시간동안 수신되지 않으면 보간하지 않고 na를 채운다
          for f in features:
            one_row[f] = np.nan
        else:             # 1시간 간격을 어긋난 수준에서 비어있다면 보간하여 데이터를 채운다
          m = t - nxt + 1
          for f in features:
            one_row[f] = 1/m * one_row[f] + (t - nxt)/m * before_row[f]
          # 보간 후 구간 유량보정값을 올바르게 한번 더 수정한다.
          one_row['구간유량보정값'] = one_row['적산유량보정값']-before_row['적산유량보정값']
        receive_check.append(receive_cnt)
        receive_cnt = 0
        new_df = new_df.append(one_row)
      nxt += 1
  new_df.loc[:, '순번'] = list(range(0, new_df.shape[0]))
  re_ch = pd.DataFrame({'순번': new_df.loc[:, '순번'], '수신횟수':receive_check},)

  new_df = new_df.join(re_ch.set_index('순번'), on='순번')
  new_df.to_csv("사람별\\누수2_"+str(p)+".csv", sep=',', na_rep='NaN')
    
  